In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import numpy as np
import pandas as pd

In [33]:
%run "data_generator.ipynb"

In [2]:
relevant_drafts = pd.read_csv("./data/VOW_prepared.csv", index_col=0, nrows = 1)
card_names = [c.split("_")[1] for c in relevant_drafts.columns if "pool_" in c]

In [34]:
columns = list(relevant_drafts.columns[14:]) +['pack_number']
train_generator = DraftReadFileGenerator("./Data/VOW_prepared.csv", 0, 800000, 128, 128000, columns, card_names)
valid_generator = DraftReadFileGenerator("./Data/VOW_prepared.csv", 800000, 900000, 128, 128000, columns, card_names)
test_generator = DraftReadFileGenerator("./Data/VOW_prepared.csv", 900000, 1000000, 128, 128000, columns, card_names)

In [ ]:
#x = relevant_drafts[list(relevant_drafts.columns[14:]) +['pack_number']]
#y = relevant_drafts["pick"]

In [ ]:
#pick_names = [c for c in relevant_drafts.columns if "pack_card_" in c]

In [ ]:
#deck_type = ["One", "Two", "Three", "Four", "Five", "Six", "R", "G", "B", "A", "U", "W"]

In [ ]:
#x = x[pick_names + deck_type]

In [ ]:
#y_index = [card_names.index(y_name) for y_name in y]
#y = keras.utils.to_categorical(y_index, len(card_names))

In [23]:
#threshold_valid = int(len(x) * 0.8)
#threshold_test = int(len(x) * 0.9)
#x_train = x[:threshold_valid]
#x_valid = x[threshold_valid:threshold_test]

#x_test = x[threshold_test:]
#y_train = y[:threshold_valid]
#y_valid = y[threshold_valid:threshold_test]
#y_test = y[threshold_test:]

In [7]:
size = len(card_names)
input_size = len(columns)

In [37]:
model = keras.Sequential(
    [
        keras.Input(shape=(input_size)),
        layers.Dense(4*size, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
        layers.Dense(4*size, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
        #layers.Dense(5*size, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
        layers.Dense(size, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 1088)              607104    
                                                                 
 dense_13 (Dense)            (None, 1088)              1184832   
                                                                 
 dense_14 (Dense)            (None, 272)               296208    
                                                                 
Total params: 2,088,144
Trainable params: 2,088,144
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x = train_generator, epochs=epochs, validation_data = valid_generator, shuffle=False)

Epoch 1/20
6250/6250 [==============================] - 182s 29ms/step - loss: 3.0995 - accuracy: 0.4387 - val_loss: 2.4419 - val_accuracy: 0.5346
Epoch 2/20
6250/6250 [==============================] - 186s 30ms/step - loss: 2.3066 - accuracy: 0.5362 - val_loss: 2.1208 - val_accuracy: 0.5654
Epoch 3/20
6250/6250 [==============================] - 183s 29ms/step - loss: 2.0380 - accuracy: 0.5662 - val_loss: 1.9007 - val_accuracy: 0.5981
Epoch 4/20
6250/6250 [==============================] - 182s 29ms/step - loss: 1.8848 - accuracy: 0.5832 - val_loss: 1.7987 - val_accuracy: 0.6016
Epoch 5/20
6250/6250 [==============================] - 182s 29ms/step - loss: 1.7891 - accuracy: 0.5927 - val_loss: 1.6948 - val_accuracy: 0.6119
Epoch 6/20
6250/6250 [==============================] - 181s 29ms/step - loss: 1.7206 - accuracy: 0.6002 - val_loss: 1.6523 - val_accuracy: 0.6195
Epoch 7/20
6250/6250 [==============================] - 182s 29ms/step - loss: 1.6634 - accuracy: 0.6072 - val_loss: 1

In [17]:
prediction_test = model.predict(x_test)
prediction_valid = model.predict(x_valid)
prediction_train = model.predict(x_train)

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
def prob_to_cards(lst):
    return [card_names[np.argmax(x)] for x in lst]

In [20]:
print(accuracy_score(prob_to_cards(y_train), prob_to_cards(prediction_train)))
print(accuracy_score(prob_to_cards(y_valid), prob_to_cards(prediction_valid)))
print(accuracy_score(prob_to_cards(y_test), prob_to_cards(prediction_test)))

0.65515875
0.63919
0.64725


In [ ]:
def parse_pack(row):
    pack = [c for c in row.index if "pack_card_" in c]
    #print(pack)
    names = []
    for i,p in enumerate(pack): 
        if row[p] == 1:
            names.append((i,p.split("_")[2]))
    return names

In [ ]:
def shown_pack_predictions(pack_data, probabilities, index, expected, predicted):
    pack = parse_pack(pack_data.iloc[index])
    print(pack_data.iloc[index].pack_number)
    print(expected[index])
    for i,card in pack:
        i = list(clf.classes_).index(card)
        print(card+" "+str(probabilities[index][i]))

In [ ]:
card_names[np.argmax(sample_prediction[0])]

In [ ]:
shown_pack_predictions(x_test[:100],sample_prediction,0, y_test[:100], None)

In [ ]:
sample_prediction[0].max()

In [ ]:
x_test.shape

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices


In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])